## 1. Environment Prepare

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.envfiles/dev.env")

True

In [2]:
from pathlib import Path
data_path:Path = Path("../../data")

## 2. Import Modules

In [3]:
from src.db import init_db, OptoFile
from src.db._var import delete_db

import pandas as pd
from datetime import datetime, timedelta


## 3. Re-Initialize Database

In [4]:
await delete_db()
await init_db()

Database 'raman' dropped successfully.


## 4. Populate Database

In [5]:
for folder in sorted(data_path.glob("finger/*")):
    subject_id = folder.name
    f_list = sorted(folder.glob("25*"))
    print(subject_id, len(f_list))
    for f in f_list:
        optofile = OptoFile.read_opto_file(f, subject_id=subject_id)
        await optofile.save()


.DS_Store 0
s1 25
s10 26
s11 26
s12 29
s13 27
s2 26
s3 30
s4 32
s5 28
s6 27
s7 26
s8 26
s9 32


In [6]:
optofile

OptoFile(id=ObjectId('691b3cafcf94c5904485e916'), subject_id='s9', header='Data Generated by Optosky', file_version='1.2', name='2510091715389 350mw 3000ms', creator='admin', description='', created=datetime.datetime(2025, 10, 9, 5, 15, 38), integration_time=3000, laser_power=350, average_number=5, scan_mode='High Precision Scan', scan_interval=8000, device_model='ATR25000', pixel_num=2048, device_sn='242320457XX', pretreat=None, pixel=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,

## 5. Create Meta file

In [7]:
timedelta(minutes=15)

datetime.timedelta(seconds=900)

In [ ]:
# data_columns = ["created","glucose"]
# subject_id = "s13"
# dt = datetime(year=2025, month=10, day=13)
# data = []
# data.append((dt.replace(hour=9, minute=20), 89))
# start = dt.replace(hour=9, minute=25)
# data.append((start + timedelta(minutes=15) * 0, 91))
# data.append((start + timedelta(minutes=15) * 1, 146))
# data.append((start + timedelta(minutes=15) * 2, 142))
# data.append((start + timedelta(minutes=15) * 3, 129))
# data.append((start + timedelta(minutes=15) * 4, 115))
# data.append((start + timedelta(minutes=15) * 5, 115))
# data.append((start + timedelta(minutes=15) * 6, 122))
# data.append((start + timedelta(minutes=15) * 7, 127))
# data.append((start + timedelta(minutes=15) * 8, 118))

# pd.DataFrame(columns=data_columns, data=data).set_index("created").to_csv(data_path.joinpath(f"{subject_id}.csv"))

In [ ]:
# print(subject_id)
# pd.read_csv(data_path.joinpath(f"{subject_id}.csv"), index_col="created", parse_dates=["created"])

s13


,glucose
created,
2025-10-13 09:20:00,89
2025-10-13 09:25:00,91
2025-10-13 09:40:00,146
2025-10-13 09:55:00,142
2025-10-13 10:10:00,129
2025-10-13 10:25:00,115
2025-10-13 10:40:00,115
2025-10-13 10:55:00,122
2025-10-13 11:10:00,127


## 6. Fetch data from DB and Map target Glucose

In [ ]:
subject_ids = [
    "s1", "s2", "s3", "s4", "s5", "s6", "s7",
    "s8", "s9", "s10","s11","s12","s13"
]

async def map_glucose_target(subject_id:str):
    # fetch samples from DB
    samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
    # read csv data
    df = pd.read_csv(data_path.joinpath(f"{subject_id}.csv"), index_col="created", parse_dates=["created"])

    
    for sample in samples:
        df['diff'] = (df.index - sample.created)
        position = df['diff'].dt.total_seconds().abs().argmin()
        diff = df.iat[position, df.columns.get_loc('diff')]
        if(abs(diff) > timedelta(minutes=3)): continue

        glucose = float(df.iat[position, df.columns.get_loc('glucose')])
        sample.glucose_target = glucose
        sample.prick_time = df.index[position]
        await sample.save()
        print(sample.subject_id, sample.created, sample.prick_time, sample.glucose_target, diff, sep="|")


for subject_id in subject_ids:
    await map_glucose_target(subject_id=subject_id)

s1|2025-09-25 08:16:44|2025-09-25 08:15:00|90.0|-1 days +23:58:16
s1|2025-09-25 08:20:57|2025-09-25 08:20:00|112.0|-1 days +23:59:03
s1|2025-09-25 08:25:29|2025-09-25 08:25:00|132.0|-1 days +23:59:31
s1|2025-09-25 08:31:10|2025-09-25 08:30:00|145.0|-1 days +23:58:50
s1|2025-09-25 08:36:05|2025-09-25 08:35:00|175.0|-1 days +23:58:55
s1|2025-09-25 08:42:30|2025-09-25 08:40:00|172.0|-1 days +23:57:30
s1|2025-09-25 08:45:50|2025-09-25 08:45:00|153.0|-1 days +23:59:10
s1|2025-09-25 08:50:48|2025-09-25 08:50:00|130.0|-1 days +23:59:12
s1|2025-09-25 08:56:16|2025-09-25 08:55:00|147.0|-1 days +23:58:44
s1|2025-09-25 09:00:53|2025-09-25 09:00:00|128.0|-1 days +23:59:07
s1|2025-09-25 09:05:43|2025-09-25 09:05:00|120.0|-1 days +23:59:17
s1|2025-09-25 09:10:48|2025-09-25 09:10:00|123.0|-1 days +23:59:12
s1|2025-09-25 09:15:55|2025-09-25 09:15:00|129.0|-1 days +23:59:05
s1|2025-09-25 09:21:04|2025-09-25 09:20:00|130.0|-1 days +23:58:56
s1|2025-09-25 09:26:12|2025-09-25 09:25:00|119.0|-1 days +23:58

## 7. Manual Check and Patch


In [7]:
subject_id = "s1"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
# df = pd.read_csv(data_path.joinpath(f"{subject_id}.csv"), index_col="created", parse_dates=["created"])

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-09-25 08:16:44|2025-09-25 08:15:00|90.0
1|2025-09-25 08:20:57|2025-09-25 08:20:00|112.0
2|2025-09-25 08:25:29|2025-09-25 08:25:00|132.0
3|2025-09-25 08:31:10|2025-09-25 08:30:00|145.0
4|2025-09-25 08:36:05|2025-09-25 08:35:00|175.0
5|2025-09-25 08:42:30|2025-09-25 08:40:00|172.0
6|2025-09-25 08:45:50|2025-09-25 08:45:00|153.0
7|2025-09-25 08:50:48|2025-09-25 08:50:00|130.0
8|2025-09-25 08:56:16|2025-09-25 08:55:00|147.0
9|2025-09-25 09:00:53|2025-09-25 09:00:00|128.0
10|2025-09-25 09:05:43|2025-09-25 09:05:00|120.0
11|2025-09-25 09:10:48|2025-09-25 09:10:00|123.0
12|2025-09-25 09:15:55|2025-09-25 09:15:00|129.0
13|2025-09-25 09:21:04|2025-09-25 09:20:00|130.0
14|2025-09-25 09:26:12|2025-09-25 09:25:00|119.0
15|2025-09-25 09:30:54|2025-09-25 09:30:00|113.0
16|2025-09-25 09:36:12|2025-09-25 09:35:00|119.0
17|2025-09-25 09:40:55|2025-09-25 09:40:00|115.0
18|2025-09-25 09:45:55|2025-09-25 09:45:00|112.0
19|2025-09-25 09:50:37|2025-09-25 09:50:00|115.0
20|2025-09-25 09:56:01|2025-09-

In [8]:
subject_id = "s2"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
# df = pd.read_csv(data_path.joinpath(f"{subject_id}.csv"), index_col="created", parse_dates=["created"])

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-09-27 08:42:30|2025-09-27 08:42:00|82.0
1|2025-09-27 08:50:08|2025-09-27 08:50:00|87.0
2|2025-09-27 08:55:41|None|None
3|2025-09-27 09:00:44|None|None
4|2025-09-27 09:06:05|2025-09-27 09:05:00|139.0
5|2025-09-27 09:10:34|None|None
6|2025-09-27 09:15:44|None|None
7|2025-09-27 09:21:54|2025-09-27 09:20:00|158.0
8|2025-09-27 09:25:47|None|None
9|2025-09-27 09:30:49|None|None
10|2025-09-27 09:36:32|2025-09-27 09:35:00|147.0
11|2025-09-27 09:40:44|None|None
12|2025-09-27 09:45:31|None|None
13|2025-09-27 09:51:19|2025-09-27 09:50:00|136.0
14|2025-09-27 09:55:19|None|None
15|2025-09-27 10:00:03|None|None
16|2025-09-27 10:06:29|2025-09-27 10:05:00|124.0
17|2025-09-27 10:10:45|None|None
18|2025-09-27 10:15:08|None|None
19|2025-09-27 10:21:19|2025-09-27 10:20:00|119.0
20|2025-09-27 10:25:31|None|None
21|2025-09-27 10:29:54|None|None
22|2025-09-27 10:37:11|2025-09-27 10:35:00|113.0
23|2025-09-27 10:40:59|None|None
24|2025-09-27 10:45:04|None|None
25|2025-09-27 10:50:57|2025-09-27 10:50:00|

In [9]:
subject_id = "s3"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
# df = pd.read_csv(data_path.joinpath(f"{subject_id}.csv"), index_col="created", parse_dates=["created"])
samples[0].glucose_target = 87
samples[0].prick_time = datetime(year=2025, month=10, day=1, hour=9, minute=20)
await samples[0].save()

samples[1].glucose_target = 87
samples[1].prick_time = datetime(year=2025, month=10, day=1, hour=9, minute=20)
await samples[1].save()

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-01 09:23:21|2025-10-01 09:20:00|87.0
1|2025-10-01 09:24:15|2025-10-01 09:20:00|87.0
2|2025-10-01 09:28:33|2025-10-01 09:27:00|88.0
3|2025-10-01 09:33:37|None|None
4|2025-10-01 09:37:42|None|None
5|2025-10-01 09:43:16|2025-10-01 09:42:00|122.0
6|2025-10-01 09:47:32|None|None
7|2025-10-01 09:52:24|None|None
8|2025-10-01 09:58:06|2025-10-01 09:57:00|144.0
9|2025-10-01 10:02:35|None|None
10|2025-10-01 10:07:36|None|None
11|2025-10-01 10:13:03|2025-10-01 10:12:00|164.0
12|2025-10-01 10:17:47|None|None
13|2025-10-01 10:22:20|None|None
14|2025-10-01 10:27:58|2025-10-01 10:27:00|154.0
15|2025-10-01 10:32:47|None|None
16|2025-10-01 10:37:28|None|None
17|2025-10-01 10:42:51|2025-10-01 10:42:00|118.0
18|2025-10-01 10:47:40|None|None
19|2025-10-01 10:52:32|None|None
20|2025-10-01 10:57:53|2025-10-01 10:57:00|107.0
21|2025-10-01 11:02:39|None|None
22|2025-10-01 11:07:30|None|None
23|2025-10-01 11:13:00|2025-10-01 11:12:00|121.0
24|2025-10-01 11:17:02|None|None
25|2025-10-01 11:22:27|None|

In [10]:
subject_id = "s4"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-02 08:50:18|2025-10-02 08:50:00|85.0
1|2025-10-02 08:55:21|2025-10-02 08:55:00|92.0
2|2025-10-02 09:00:05|None|None
3|2025-10-02 09:04:59|None|None
4|2025-10-02 09:10:38|2025-10-02 09:10:00|141.0
5|2025-10-02 09:14:54|None|None
6|2025-10-02 09:20:01|None|None
7|2025-10-02 09:25:43|2025-10-02 09:25:00|166.0
8|2025-10-02 09:29:56|None|None
9|2025-10-02 09:35:05|None|None
10|2025-10-02 09:40:35|2025-10-02 09:40:00|186.0
11|2025-10-02 09:44:54|None|None
12|2025-10-02 09:49:55|None|None
13|2025-10-02 09:56:01|2025-10-02 09:55:00|175.0
14|2025-10-02 09:59:51|None|None
15|2025-10-02 10:05:07|None|None
16|2025-10-02 10:11:00|2025-10-02 10:10:00|170.0
17|2025-10-02 10:14:47|None|None
18|2025-10-02 10:19:56|None|None
19|2025-10-02 10:25:50|2025-10-02 10:25:00|151.0
20|2025-10-02 10:30:05|None|None
21|2025-10-02 10:34:53|None|None
22|2025-10-02 10:40:47|2025-10-02 10:40:00|143.0
23|2025-10-02 10:44:53|None|None
24|2025-10-02 10:49:45|None|None
25|2025-10-02 10:55:47|2025-10-02 10:55:00|

In [11]:
subject_id = "s5"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-04 10:06:59|2025-10-04 10:05:00|89.0
1|2025-10-04 10:11:44|2025-10-04 10:10:00|91.0
2|2025-10-04 10:15:03|None|None
3|2025-10-04 10:20:01|None|None
4|2025-10-04 10:25:22|2025-10-04 10:25:00|123.0
5|2025-10-04 10:29:58|None|None
6|2025-10-04 10:35:13|None|None
7|2025-10-04 10:40:48|2025-10-04 10:40:00|180.0
8|2025-10-04 10:45:08|None|None
9|2025-10-04 10:50:15|None|None
10|2025-10-04 10:55:32|2025-10-04 10:55:00|191.0
11|2025-10-04 10:59:56|None|None
12|2025-10-04 11:05:14|None|None
13|2025-10-04 11:10:39|2025-10-04 11:10:00|189.0
14|2025-10-04 11:15:03|None|None
15|2025-10-04 11:20:16|None|None
16|2025-10-04 11:26:06|2025-10-04 11:25:00|179.0
17|2025-10-04 11:26:48|2025-10-04 11:25:00|179.0
18|2025-10-04 11:27:29|2025-10-04 11:25:00|179.0
19|2025-10-04 11:30:08|None|None
20|2025-10-04 11:35:27|None|None
21|2025-10-04 11:41:20|2025-10-04 11:40:00|160.0
22|2025-10-04 11:45:39|None|None
23|2025-10-04 11:50:31|None|None
24|2025-10-04 11:56:07|2025-10-04 11:55:00|166.0
25|2025-10-

In [12]:
subject_id = "s6"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-05 09:56:21|2025-10-05 09:55:00|83.0
1|2025-10-05 10:06:06|2025-10-05 10:05:00|113.0
2|2025-10-05 10:10:04|None|None
3|2025-10-05 10:15:14|None|None
4|2025-10-05 10:20:52|2025-10-05 10:20:00|192.0
5|2025-10-05 10:25:29|None|None
6|2025-10-05 10:29:51|None|None
7|2025-10-05 10:36:10|2025-10-05 10:35:00|181.0
8|2025-10-05 10:40:00|None|None
9|2025-10-05 10:45:04|None|None
10|2025-10-05 10:51:11|2025-10-05 10:50:00|141.0
11|2025-10-05 10:55:17|None|None
12|2025-10-05 11:00:32|None|None
13|2025-10-05 11:05:38|2025-10-05 11:05:00|153.0
14|2025-10-05 11:10:24|None|None
15|2025-10-05 11:11:08|None|None
16|2025-10-05 11:15:29|None|None
17|2025-10-05 11:21:09|2025-10-05 11:20:00|181.0
18|2025-10-05 11:25:23|None|None
19|2025-10-05 11:29:54|None|None
20|2025-10-05 11:36:16|2025-10-05 11:35:00|192.0
21|2025-10-05 11:40:15|None|None
22|2025-10-05 11:45:07|None|None
23|2025-10-05 11:50:55|2025-10-05 11:50:00|190.0
24|2025-10-05 11:55:11|None|None
25|2025-10-05 11:59:58|None|None
26|2025-1

In [13]:
subject_id = "s7"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-07 06:41:43|2025-10-07 06:40:00|84.0
1|2025-10-07 06:51:22|2025-10-07 06:50:00|265.0
2|2025-10-07 06:55:07|None|None
3|2025-10-07 07:00:01|None|None
4|2025-10-07 07:06:23|2025-10-07 07:05:00|151.0
5|2025-10-07 07:10:03|None|None
6|2025-10-07 07:14:59|None|None
7|2025-10-07 07:21:28|2025-10-07 07:20:00|179.0
8|2025-10-07 07:25:24|None|None
9|2025-10-07 07:31:24|None|None
10|2025-10-07 07:35:54|2025-10-07 07:35:00|196.0
11|2025-10-07 07:39:52|None|None
12|2025-10-07 07:45:06|None|None
13|2025-10-07 07:51:27|2025-10-07 07:50:00|132.0
14|2025-10-07 07:55:02|None|None
15|2025-10-07 08:00:53|None|None
16|2025-10-07 08:06:03|2025-10-07 08:05:00|145.0
17|2025-10-07 08:10:14|None|None
18|2025-10-07 08:14:58|None|None
19|2025-10-07 08:21:03|2025-10-07 08:20:00|161.0
20|2025-10-07 08:25:08|None|None
21|2025-10-07 08:29:58|None|None
22|2025-10-07 08:36:16|2025-10-07 08:35:00|153.0
23|2025-10-07 08:40:11|None|None
24|2025-10-07 08:45:10|None|None
25|2025-10-07 08:51:46|2025-10-07 08:50:00

In [14]:
subject_id = "s8"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
samples[0].glucose_target = 92
samples[0].prick_time = datetime(year=2025, month=10, day=8, hour=10, minute=45)
await samples[0].save()
for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-08 10:47:36|2025-10-08 10:45:00|92.0
1|2025-10-08 10:50:49|2025-10-08 10:50:00|99.0
2|2025-10-08 10:55:33|None|None
3|2025-10-08 11:00:43|None|None
4|2025-10-08 11:07:41|2025-10-08 11:05:00|156.0
5|2025-10-08 11:10:00|None|None
6|2025-10-08 11:14:52|None|None
7|2025-10-08 11:20:31|2025-10-08 11:20:00|165.0
8|2025-10-08 11:25:05|None|None
9|2025-10-08 11:29:52|None|None
10|2025-10-08 11:36:04|2025-10-08 11:35:00|136.0
11|2025-10-08 11:39:50|None|None
12|2025-10-08 11:44:45|None|None
13|2025-10-08 11:50:10|2025-10-08 11:50:00|133.0
14|2025-10-08 11:55:14|None|None
15|2025-10-08 12:00:13|None|None
16|2025-10-08 12:05:31|2025-10-08 12:05:00|125.0
17|2025-10-08 12:09:55|None|None
18|2025-10-08 12:15:22|None|None
19|2025-10-08 12:20:48|2025-10-08 12:20:00|135.0
20|2025-10-08 12:25:29|None|None
21|2025-10-08 12:29:55|None|None
22|2025-10-08 12:36:21|2025-10-08 12:35:00|132.0
23|2025-10-08 12:40:04|None|None
24|2025-10-08 12:45:08|None|None
25|2025-10-08 12:50:45|2025-10-08 12:50:00|

In [15]:
subject_id = "s9"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-09 02:39:31|2025-10-09 02:42:00|96.0
1|2025-10-09 02:43:16|2025-10-09 02:42:00|96.0
2|2025-10-09 02:47:38|None|None
3|2025-10-09 02:52:38|None|None
4|2025-10-09 02:57:36|2025-10-09 02:57:00|165.0
5|2025-10-09 03:02:38|None|None
6|2025-10-09 03:07:38|None|None
7|2025-10-09 03:13:15|2025-10-09 03:12:00|126.0
8|2025-10-09 03:17:52|None|None
9|2025-10-09 03:22:39|None|None
10|2025-10-09 03:27:48|2025-10-09 03:27:00|111.0
11|2025-10-09 03:33:02|None|None
12|2025-10-09 03:37:40|None|None
13|2025-10-09 03:42:40|2025-10-09 03:42:00|139.0
14|2025-10-09 03:47:53|None|None
15|2025-10-09 03:52:40|None|None
16|2025-10-09 03:57:57|2025-10-09 03:57:00|160.0
17|2025-10-09 04:03:06|None|None
18|2025-10-09 04:08:28|None|None
19|2025-10-09 04:13:40|2025-10-09 04:12:00|148.0
20|2025-10-09 04:18:49|None|None
21|2025-10-09 04:23:39|None|None
22|2025-10-09 04:28:51|2025-10-09 04:27:00|134.0
23|2025-10-09 04:33:38|None|None
24|2025-10-09 04:39:06|None|None
25|2025-10-09 04:45:23|2025-10-09 04:45:00|

In [16]:
subject_id = "s10"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)
samples[0].glucose_target = 98
samples[0].prick_time = datetime(year=2025, month=10, day=10, hour=9, minute=50)
await samples[0].save()
for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-10 09:53:12|2025-10-10 09:50:00|98.0
1|2025-10-10 10:01:31|2025-10-10 10:00:00|111.0
2|2025-10-10 10:05:42|None|None
3|2025-10-10 10:09:45|None|None
4|2025-10-10 10:15:35|2025-10-10 10:15:00|171.0
5|2025-10-10 10:20:14|None|None
6|2025-10-10 10:25:42|None|None
7|2025-10-10 10:30:48|2025-10-10 10:30:00|195.0
8|2025-10-10 10:35:19|None|None
9|2025-10-10 10:39:43|None|None
10|2025-10-10 10:46:56|2025-10-10 10:45:00|205.0
11|2025-10-10 10:50:04|None|None
12|2025-10-10 10:55:20|None|None
13|2025-10-10 11:00:59|2025-10-10 11:00:00|184.0
14|2025-10-10 11:05:13|None|None
15|2025-10-10 11:10:17|None|None
16|2025-10-10 11:16:51|2025-10-10 11:15:00|171.0
17|2025-10-10 11:19:58|None|None
18|2025-10-10 11:25:17|None|None
19|2025-10-10 11:31:22|2025-10-10 11:30:00|154.0
20|2025-10-10 11:35:22|None|None
21|2025-10-10 11:39:58|None|None
22|2025-10-10 11:47:18|2025-10-10 11:45:00|147.0
23|2025-10-10 11:49:59|None|None
24|2025-10-10 11:55:42|None|None
25|2025-10-10 12:01:19|2025-10-10 12:00:00

In [17]:
subject_id = "s11"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-11 09:25:10|2025-10-11 09:25:00|117.0
1|2025-10-11 09:31:11|2025-10-11 09:30:00|118.0
2|2025-10-11 09:35:17|None|None
3|2025-10-11 09:40:29|None|None
4|2025-10-11 09:46:12|2025-10-11 09:45:00|153.0
5|2025-10-11 09:50:25|None|None
6|2025-10-11 09:55:05|None|None
7|2025-10-11 10:01:35|2025-10-11 10:00:00|210.0
8|2025-10-11 10:05:50|None|None
9|2025-10-11 10:10:47|None|None
10|2025-10-11 10:15:48|2025-10-11 10:15:00|227.0
11|2025-10-11 10:20:26|None|None
12|2025-10-11 10:25:06|None|None
13|2025-10-11 10:31:22|2025-10-11 10:30:00|207.0
14|2025-10-11 10:35:27|None|None
15|2025-10-11 10:40:11|None|None
16|2025-10-11 10:46:43|2025-10-11 10:45:00|210.0
17|2025-10-11 10:49:48|None|None
18|2025-10-11 10:55:39|None|None
19|2025-10-11 11:01:24|2025-10-11 11:00:00|184.0
20|2025-10-11 11:05:34|None|None
21|2025-10-11 11:10:37|None|None
22|2025-10-11 11:16:12|2025-10-11 11:15:00|168.0
23|2025-10-11 11:20:33|None|None
24|2025-10-11 11:25:19|None|None
25|2025-10-11 11:31:24|2025-10-11 11:30:0

In [18]:
subject_id = "s12"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-12 09:23:32|2025-10-12 09:25:00|107.0
1|2025-10-12 09:35:11|None|None
2|2025-10-12 09:40:46|None|None
3|2025-10-12 09:45:44|2025-10-12 09:45:00|200.0
4|2025-10-12 09:50:44|None|None
5|2025-10-12 09:55:45|None|None
6|2025-10-12 10:00:49|2025-10-12 10:00:00|236.0
7|2025-10-12 10:05:50|None|None
8|2025-10-12 10:10:53|None|None
9|2025-10-12 10:16:12|2025-10-12 10:15:00|234.0
10|2025-10-12 10:20:50|None|None
11|2025-10-12 10:25:49|None|None
12|2025-10-12 10:30:58|2025-10-12 10:30:00|228.0
13|2025-10-12 10:35:53|None|None
14|2025-10-12 10:40:53|None|None
15|2025-10-12 10:43:09|None|None
16|2025-10-12 10:48:10|2025-10-12 10:48:00|218.0
17|2025-10-12 10:54:03|None|None
18|2025-10-12 10:59:44|None|None
19|2025-10-12 11:04:48|2025-10-12 11:04:00|206.0
20|2025-10-12 11:09:51|None|None
21|2025-10-12 11:14:43|None|None
22|2025-10-12 11:19:52|2025-10-12 11:19:00|196.0
23|2025-10-12 11:25:04|None|None
24|2025-10-12 11:31:49|None|None
25|2025-10-12 11:36:46|2025-10-12 11:36:00|157.0
26|2025-

In [19]:
subject_id = "s13"
samples:list[OptoFile] = await OptoFile.fetch_all(subject_id=subject_id)

for idx, sample in enumerate(samples):
    print(idx, sample.created, sample.prick_time, sample.glucose_target, sep="|")

0|2025-10-13 09:26:24|2025-10-13 09:25:00|91.0
1|2025-10-13 09:30:03|None|None
2|2025-10-13 09:35:06|None|None
3|2025-10-13 09:40:44|2025-10-13 09:40:00|146.0
4|2025-10-13 09:44:58|None|None
5|2025-10-13 09:50:02|None|None
6|2025-10-13 09:56:21|2025-10-13 09:55:00|142.0
7|2025-10-13 09:59:57|None|None
8|2025-10-13 10:05:06|None|None
9|2025-10-13 10:06:04|None|None
10|2025-10-13 10:10:53|2025-10-13 10:10:00|129.0
11|2025-10-13 10:14:56|None|None
12|2025-10-13 10:19:54|None|None
13|2025-10-13 10:25:37|2025-10-13 10:25:00|115.0
14|2025-10-13 10:29:53|None|None
15|2025-10-13 10:30:45|None|None
16|2025-10-13 10:35:00|None|None
17|2025-10-13 10:41:07|2025-10-13 10:40:00|115.0
18|2025-10-13 10:45:19|None|None
19|2025-10-13 10:49:46|None|None
20|2025-10-13 10:55:51|2025-10-13 10:55:00|122.0
21|2025-10-13 10:59:59|None|None
22|2025-10-13 11:05:13|None|None
23|2025-10-13 11:11:40|2025-10-13 11:10:00|127.0
24|2025-10-13 11:15:29|None|None
25|2025-10-13 11:19:54|None|None
26|2025-10-13 11:25:57|20